# Risk profiling for customer churn

In [6]:
#https://www.kaggle.com/osmi/mental-health-in-tech-survey

Some of my own findings + exploration page!!

To note:
- remove self-emplyed
- measure male vs. female vs other
- have you sought treatments? 


- How does remote work affect your mental health?
- How do big vs. small companies compare
- if you are self-employed or not
- seek help - most companies don't provide help or employees don't know if they those options

To explore:
- what is a stronger predictor: location, support by employer, remote work, size of the company, family history, etc. 
Sankey: 
- does it make a difference if an employer just has benefits, or is active in supporting the mental health? 
- out of those where mental health is supported/not at work->  does it interfere with your work -> how many have seeked treatment? 

In [7]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
from pandas_profiling import ProfileReport
import numpy as np

In [8]:
# Use the Kaggle API to download the dataset
download = False

if download: 
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files('osmihelp/osmi-mental-health-in-tech-survey-2019', path='kaggle_datasets', unzip=True)

In [9]:
survey_df = pd.read_csv('kaggle_datasets/OSMI 2019 Mental Health in Tech Survey Results - OSMI Mental Health in Tech Survey 2019.csv')

In [10]:
survey_df.shape

(352, 82)

In [11]:
basic_list = ['male', 'female']
column_main = 'What is your gender?'

def clean_gender_data(df):
    df[column_main] = df[column_main].str.lower()
    df[column_main] = df[column_main].replace(['m', 'cis male', 'male ', 'man', 'cis man',
                                         'male (cis)', 'msle', 'make', 'mail', 'malr', 'maile'], 'male')
    df[column_main] = df[column_main].replace(['female', 'woman', 'female ', 
                                         'f', 'female (cis)', 'femake', 'femail', 'femile', 'femmina',
                                              'cis woman', ], 'female')
    
    df[column_main].loc[~df[column_main].isin(basic_list)]='other'
    return df

survey_df = clean_gender_data(survey_df)

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
survey_df['What is your gender?'].value_counts()

male      225
female     99
other      28
Name: What is your gender?, dtype: int64

## Get age groups

In [13]:
# Getting rid of 28 rows
survey_df = survey_df[survey_df['What is your age?']!='other']
survey_df['What is your age?'] = survey_df['What is your age?'].astype(np.int)
survey_df['What is your age?'].value_counts()

def get_age_groups(row):
        if (row['What is your age?'] >= 20) & (row['What is your age?'] < 25):
            return '20-25'
        elif (row['What is your age?'] >= 25) & (row['What is your age?'] < 30):
            return '25-30'
        elif (row['What is your age?'] >= 30) & (row['What is your age?'] < 35):
            return '30-35'   
        elif (row['What is your age?'] >= 35) & (row['What is your age?'] < 40):
            return '35-40'          
        elif (row['What is your age?'] >= 40) & (row['What is your age?'] < 45):
            return '40-45'         
        elif (row['What is your age?'] >= 45) & (row['What is your age?'] < 50):
            return '45-50'         
        elif (row['What is your age?'] >= 50) & (row['What is your age?'] < 55):
            return '50-55'   
        elif (row['What is your age?'] >= 55) & (row['What is your age?'] < 60):
            return '55-60'   
        elif (row['What is your age?'] >= 60) & (row['What is your age?'] < 65):
            return '60-65'
        else:
            return 'other'
        
survey_df['age_group'] = survey_df.apply(get_age_groups, axis=1)

/usr/local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [14]:
survey_df.shape

(352, 83)

Get age groups in percentage

In [15]:
survey_df = survey_df.rename(columns={'Why or why not?.1':'Would you bring up your mental health at a job interview - why or why not?'}) 

In [16]:
survey_df.to_csv('clean_mental_health_data.csv')